<a href="https://colab.research.google.com/github/Rainniee/Neural-Networks-AI/blob/master/Experiment4_H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division
from __future__ import print_function

import pandas as pd
import numpy as np

import itertools
import os

import nltk
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder


%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.datasets import imdb
from tensorflow import keras
from keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM, Activation, Dropout
from keras.layers import LSTM, CuDNNLSTM

from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing import text, sequence
from keras import utils
from tensorflow.keras.callbacks import EarlyStopping

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

from distutils.version import LooseVersion as LV
from keras import __version__
from keras import backend as K

import seaborn as sns
sns.set()

Using TensorFlow backend.


You have TensorFlow version 1.13.1


In [3]:
!pip install h2o

    100% |████████████████████████████████| 121.0MB 289kB/s 
  Stored in directory: /root/.cache/pip/wheels/ca/0a/e6/f0538dc7e67f9c76b5c93dcc3529c1a00adeed4dacd31d551a
Successfully built h2o


In [4]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5f2p61ab
  JVM stdout: /tmp/tmp5f2p61ab/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5f2p61ab/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,4 hours and 52 minutes
H2O cluster name:,H2O_from_python_unknownUser_qwrmw4
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [5]:
from google.colab import files
upload = files.upload()

Saving cleanedfinancial_data.csv to cleanedfinancial_data (1).csv


In [6]:
h2o_data = h2o.import_file("cleanedfinancial_data.csv")
h2o_data.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2
sentiment,text
neutral,"Good day, ladies and gentlemen, and welcome to the Tesla, Inc. Q4 2018 Financial Results and Q&A Webcast. At this time, all participants are in a listen-only mode. Later, we will conduct a question-and-answer session, and instructions will follow at that time. [Operator Instructions] As a reminder, this conference is being recorded."
negative,"I'm not sure. I think Model T was a little bit slower, but I'm not 100% sure."
negative,"Well, we need to bring the Shanghai factory online. I think that's the biggest driver for getting to 500K plus a year. Our car is just very expensive going into China. We've got import duties. We've got transport costs. We've got higher-cost labor here. And we've never been eligible for any of the EV tax credits. A lot of people sort of dependent on incentives. In fact, we are [indiscernible] EVs, we have the least access to incentives. It's pretty crazy because there's so many companies that - countries that have put price caps on the EV incentive, which affects Tesla. And in China, which is the biggest market for EVs, we've never had any subsidies or tax incentives for vehicles."
neutral,"So it's - it is eligible for that. But it sounds like that's going to be reducing in China in the coming years. But, really, bottom line is, we need the Shanghai factory to achieve that 10k rate and other cars be affordable."
positive,"The demand for - the demand for Model 3 is insanely high. The inhibitor is affordability. It's just like people literally don't have the money to buy the car. It's got nothing to do with desire. They just don't have enough money in their bank account. If the car can be made more affordable, the demand is extraordinary."
neutral,Our next question comes from Pierre Ferragu with New Street Research.
neutral,"So, Deepak, I was wondering, so as you get to 2019, we're all concerned about the potential recession, and I was wondering how you think about it and what you would tell us about what we should expect - how we should expect Tesla to react to recession in 2019. How do you manage your volume land? How do you manage your pricing? How do you present cash? How do you manage your CapEx if things turn south in 2019? And then I have a follow-up on gross margin for Jerome."
positive,"Yes, it's a very broad question, which is not really just for me to answer. But I think at the highest level, the way we are trying to be prepared for any kind of contingency here is to just continue focusing on cost. And the theme of our conversations here is, how do we reduce cost all the time? And how do we run our business with a very high level of financial discipline? And Elon alluded to that and so did Jerome - Jerome, I think. That - if we do that, we believe that even in some of the scenarios of lower volumes and pricing - tight pricing, we do have a good chance and a good shot of being profitable and generate free cash flow. So that's the best way to manage the business, be frugal."
positive,"Yes, I don't want to be a broken record about this, it's costs, costs, costs, because reducing our costs - by the way, while making improvements to Model 3, I want to emphasize, the product is getting better by slight degrees despite lower costs in hundreds of small ways. But you actually wouldn't notice explicitly, but they would appreciate subconsciously. And getting those costs down, variable costs and fixed costs, is what allows us to lower the price and be financially sustainable and achieve our mission of environmental sustainability. So we have to be absolute clear about this. There's no question."


In [13]:
# Train the model
from h2o.automl import H2OAutoML

X = h2o_data["text"]
y = h2o_data["sentiment"]

h2o_data[X] = h2o_data[X].asfactor()
h2o_data[y] = h2o_data[y].asfactor()

aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=X, y=y, training_frame=h2o_data)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default

H2OResponseError: ignored